# Experimentando com bancos de dados em Python

## Conectando com a base de dados

Existem sistemas de gerenciamento de bancos de dados que operam no mesmo processo da aplicação cliente - ou seja, são bibliotecas que você pode importar diretamente no seu programa. Por exemplo: a biblioteca ``sqlite3`` (https://docs.python.org/3/library/sqlite3.html) é parte do conjunto de bibliotecas padrão da linguagem Python, e permite criar bases de dados diretamente em arquivo ou memória, sem a necessidade de um programa em separado. Eis um exemplo de uso de sqlite3:

In [1]:
import sqlite3

db = sqlite3.connect('teste.db')
# Para criar a base em memória (base temporária) use o nome de arquivo
# ":memory:"

Note que o arquivo ``teste.db`` foi criado. 

In [2]:
import os

print('teste.db' in os.listdir())

True


Apesar do ``sqlite3`` criar o sistema de gerenciamento de banco de dados (SGBD) SQLite diretamente no nosso processo (neste caso, o kernel Python que está sendo executado por esse notebook), o objeto de acesso ainda é uma "conexão". Isso se deve ao fato de que SGBDs "normais" - aqueles que realmente usamos em produção - são geralmente *sistemas cliente-servidor*, onde o SGBD é um processo separado da aplicação cliente e a comunicação entre eles acontece através de uma conexão de rede.

<center><img src='imgs/client-server.png'></center>

**Atividade:**

Na figura acima, quem é o cliente do sistema de gerenciamento de banco de dados?

**R:** É quem faz a requisição através da conexão internet, nesse caso é o microsserviço 1.

Uma conexão é simplesmente o conduíte entre o cliente e o SGBD. Em Python, a especificação PEP 249 (https://www.python.org/dev/peps/pep-0249/) define que uma conexão deve ter os métodos ``.close()`` (para fechar a conexão), ``.cursor()`` (para obter um *cursor*, que é o objeto para realmente interagir com o banco de dados), e o par ``.commit()``/``.rollback()`` (para gerenciamento de transações).

Vamos fechar essa conexão e trabalhar com o MySQL daqui em diante.

In [3]:
db.close()

In [4]:
os.remove('teste.db')

Antes de tudo vamos criar um banco de dados. Queremos criar um banco de dados para gerenciar as equipes do primeiro campeonato Insper de ninja-cowboy-urso!

- Ninja, com sua velocidade e furtividade, detona o cowboy.
- O cowboy atira no urso.
- O urso dá uma patada no ninja e acaba com ele.

Cada jogador tem um nome, uma equipe, e sua jogada favorita. As equipes tem um nome e um grito de guerra.

Portanto, nosso schema será:

```
Jogadores (id PK, nome_equipe, nome, preferencia)
Equipes (nome PK, grito)
```

com a restrição:

```
Jogadores(nome_equipe) FK para Equipes(nome)
```

E um dicionário de dados simples para este problema é:

<center>Jogadores</center>
<table>
    <tr>
        <th>Campo</th>
        <th>Tipo</th>
        <th>Significado</th>
        <th>Chave?</th>
        <th>Valores</th>
    </tr>
    <tr>
        <td>id</td>
        <td>int</td>
        <td>Identificador unico do jogador</td>
        <td>PK</td>
        <td>sem restrições</td>
    </tr>
    <tr>
        <td>nome_equipe</td>
        <td>string</td>
        <td>Nome da equipe na qual o jogador atua</td>
        <td>FK Equipes(nome)</td>
        <td></td>
    </tr>
    <tr>
        <td>nome</td>
        <td>string</td>
        <td>Nome do jogador</td>
        <td></td>
        <td>Pode haver dois jogadores com mesmo nome. Máximo 80 caracteres.</td>
    </tr>
    <tr>
        <td>preferencia</td>
        <td>int</td>
        <td>Jogada preferida do jogador</td>
        <td></td>
        <td>0: Ninja, 1: Cowboy, 2: Urso</td>
    </tr>
</table>

<center>Equipes</center>
<table>
    <tr>
        <th>Campo</th>
        <th>Tipo</th>
        <th>Significado</th>
        <th>Chave?</th>
        <th>Valores</th>
    </tr>
    <tr>
        <td>nome</td>
        <td>string</td>
        <td>Nome da equipe</td>
        <td>PK</td>
        <td>Máximo 30 caracteres</td>
    </tr>
    <tr>
        <td>grito</td>
        <td>string</td>
        <td>Grito de guerra da equipe</td>
        <td></td>
        <td>Maximo 80 caracteres</td>
    </tr>
</table>

**Atividade:** Escreva e rode o script de criação do banco de dados. Chame o banco de dados de ``torneio``. Compare com o gabarito em ``script_001.sql``.

Para conectar com o MySQL precisamos da biblioteca *mysql-connector-python*. 
Para instalar esta biblioteca:

**`pip install mysql-connector-python`**

In [5]:
import mysql.connector

Vamos agora abrir uma conexão com o MySQL. Como agora temos realmente um sistema cliente-servidor, precisamos definir onde está o servidor, e precisamos também de informações de autenticação.

In [6]:
connection_options = {
    'host': 'localhost',
    'user': 'megadados',
    'password': 'megadados',
    'database': 'torneio'
}

**Pergunta**: Faz sentido guardar o *user* e *password* dentro do próprio código fonte? Qual o jeito correto de lidar com username e password no seu produto?

**Resposta**: Não, pois assim qualquer um que tivesse acesso ao código fonte poderia acessar o banco de dados. O user e o password deveriam ser algo mais restrito, como um sistema de autenticação e senha.

- Use um arquivo de configuração (tipo, secrets.json) na conta que será usada para rodar a aplicação e isso não vai no git, ele vai no .gitignore. Este arquivo deve ter todas as permissões de acesso negadas, exceto permissão de leitura para o próprio usuário da conta de execução da aplicação. O arquivo deverá ser lido então pela aplicação no momento da inicialização.

- Use uma variável de ambiente com essas credenciais. É a mesma coisa que no ítem anterior, só ao invés de ter um arquivo de configuração da aplicação, temos o arquivo de configuração da conta em si, com os mesmos requisitos em relação às restrições de permissão.

Abrindo a conexão:

In [7]:
# A notação de duplo asterisco em Python permite usar um dicionário no lugar de argumentos nomeados.
# A linha a ser executada abaixo equivale á:
# connection = mysql.connector.connect(
#     host='localhost',
#     user='megadados',
#     password='megadados2019',
#     database='torneio'
# )
connection = mysql.connector.connect(**connection_options) # quando é lista, uma * só

**IMPORTANTE:** TODAS AS OPERAÇÕES REALIZADAS NESTA CONEXÃO SÃO TEMPORÁRIAS, ATÉ QUE O COMANDO `connection.commit()` seja executado!

**MAIS IMPORTANTE AINDA:** ISSO NÃO VALE SE `AUTOCOMMIT` ESTIVER LIGADO!

Estudaremos melhor essa questão quando discutirmos transações em maiores detalhes.

Vamos criar alguns dados iniciais:

In [8]:
T1 = 'Raposas Nerds'
T2 = 'É nois'

NINJA = 0
COWBOY = 1
URSO = 2

times = {
    T1: 'sudo vencer!',
    T2: 'Olha eu mamãe!',
}

jogadores = [
    ('Raul Ayres', T1, NINJA),
    ('Luciano Hashimoto', T1, COWBOY),
    ('Rafael Montaigner', T1, URSO),
    ('Igor Miranda', T2, URSO),
    ('Andrew Ikeda', T2, COWBOY),
    ('Fábio Kurauchi', T2, NINJA),
]

Vamos inserir os dados dos times.

**Atividade:** Quem devemos inserir primeiro: dados na tabela `Equipes` ou na tabela `Jogadores`?

**R:** Dados na tabela Equipes, isso porque em Jogadores há o campo nome_equipe que é chave estrangeira. Assim, esse campo depende de Equipes para existir, se não há equipe, esse campo fica nulo em Jogadores.


Quando uma tabela tem chave estrangeira para outra tabela dizemos que ela é filha (*child*) da outra tabela, e que a outra tabela é a pai (*parent*).

Um *cursor* é o objeto que permite executar queries SQL e interagir com os resultados obtidos.

In [9]:
try:
    with connection.cursor() as cursor:
        # Outra opção aqui seria o uso de cursor.executemany()
        for time, grito in times.items():
            cursor.execute('INSERT INTO Equipes VALUES (%s, %s)', (time, grito))
    connection.commit()
except mysql.connector.IntegrityError as exception:
    print(f'IntegrityError: {exception}')
    connection.rollback()

Note o comando ``commit`` acima: ele serve para efetivar a operação no banco de dados. Verifique no MySQL Workbench que a operação foi bem sucedida.

Podemos tambem usar nossa conexão para verificar o estado da tabela ``Equipes``:

In [10]:
with connection.cursor() as cursor:
    cursor.execute('SELECT * FROM Equipes')
    for item in cursor:
        print(item)

('É nois', 'Olha eu mamãe!')
('Raposas Nerds', 'sudo vencer!')


**Atividade:** Tente executar novamente a inserção de dados, o que acontece?



**Resposta:** IntegrityError: 1062 (23000): Duplicate entry 'Raposas Nerds' for key 'equipes.PRIMARY'

Ou seja, acontece um erro que alega que a entrada está duplicada para a chave primária de equipes.

## Interlúdio: sanitização de entradas de dados

Por que usamos aqueles ``%s`` no comando a ser executado? Por que não simplesmente incluir diretamente os dados na *string* do comando SQL? Por exemplo:

```Python
cursor.execute('INSERT INTO Equipes VALUES ("' + time + '", "' + grito + '")')
```

Muitas vezes o resultado é o mesmo! Contudo, e SE executássemos a query abaixo...

A célula acima está desabilitada. Mas se quiser executar passe para tipo "Code" e rode. Depois feche a conexão com `connection.close()`, volte para o começo do notebook e rode tudo de novo exceto a célula mortal!

![Little Bobby Tables](imgs/exploits_of_a_mom.png)

[Exploits of a mom](https://xkcd.com/327/)


Existem jeitos de quebrar o sistema com esse tipo de ataque, onde o hacker tenta inserir código SQL mal-intencionado em formulários, na expectativa de que o programador não tenha feito a sanitização das entradas. Esse ataque chama-se *'SQL injection'*.

Fim do interlúdio, voltamos à apresentação principal.

Vamos agora inserir os jogadores. Neste caso não precisamos inserir a chave primária, pois ela é gerada automaticamente. Consequentemente temos que especificar que estamos inserindo apenas as demais colunas - temos que nomear as colunas na qual estamos inserindo os dados.

In [11]:
try:
    with connection.cursor() as cursor:
        cursor.executemany(
            'INSERT INTO Jogadores (nome, nome_equipe, preferencia) VALUES (%s, %s, %s)',
            jogadores)
    connection.commit()
except Exception as e:
    print(e)
    connection.rollback()

Vamos verificar se funcionou. Para evitar ter criar os cursores a todo momento e coletar os resultados vamos construir uma classe auxiliar, só para simplificar o código:

In [12]:
class ConnectionHelper:

    def __init__(self, connection):
        self.connection = connection

    def __call__(self, query, args=None):
        with self.connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)


db = ConnectionHelper(connection)

Note o padrão de *'dependency injection'* aqui: ao invés de criar a conexão dentro do construtor da classe ConnectionHelper, passamos um objeto 'connection' que será armazenado dentro do ConnectionHelper. Qualquer objeto 'connection' pode ser passado, de qualquer classe, desde que obedeça a interface (implícita) de um objeto de conexão ao banco de dados. Neste nosso caso, tem que ter o método `cursor()` que retorne um objeto de interação com a base de dados.

## Explorando a estrutura da base de dados

Vamos ver quais tabelas existem na base ``torneio``:

In [13]:
db('SHOW TABLES')

Executando query:
('equipes',)
('jogadores',)


Para saber qual o schema da tabela 'jogadores', podemos usar o comando '`DESCRIBE`'

In [14]:
db('DESCRIBE jogadores')

Executando query:
('id', b'int', 'NO', 'PRI', None, 'auto_increment')
('nome', b'varchar(80)', 'NO', '', None, '')
('nome_equipe', b'varchar(30)', 'NO', 'MUL', None, '')
('preferencia', b'int', 'YES', '', None, '')


## Consultando a base de dados

Vamos usar o comando `SELECT` para listar os conteudos da tabela 'Jpgadores'

In [15]:
db('SELECT * FROM Jogadores')

Executando query:
(1, 'Raul Ayres', 'Raposas Nerds', 0)
(2, 'Luciano Hashimoto', 'Raposas Nerds', 1)
(3, 'Rafael Montaigner', 'Raposas Nerds', 2)
(4, 'Igor Miranda', 'É nois', 2)
(5, 'Andrew Ikeda', 'É nois', 1)
(6, 'Fábio Kurauchi', 'É nois', 0)


O comando acima lista todos os registros da tabela `Jogadores`, com todas as colunas presentes:

![Seleção da tabela inteira](imgs/tudo.png)

Vamos agora selecionar apenas algumas colunas para exibir.

In [16]:
db('SELECT nome, nome_equipe FROM Jogadores')

Executando query:
('Raul Ayres', 'Raposas Nerds')
('Luciano Hashimoto', 'Raposas Nerds')
('Rafael Montaigner', 'Raposas Nerds')
('Igor Miranda', 'É nois')
('Andrew Ikeda', 'É nois')
('Fábio Kurauchi', 'É nois')


Agora vemos apenas as colunas escolhidas.  A operação de seleção de colunas chama-se **projeção**.

![Projeção](imgs/projecao.png)

Vamos agora atuar na escolha de linhas, selecionando quais desejamos. Para escolher todas as linhas cujo `nome_equipe` começa com 'Rap' podemos executar a query a seguir:

In [17]:
db("SELECT * FROM Jogadores WHERE nome_equipe LIKE 'Rap%'")

Executando query:
(1, 'Raul Ayres', 'Raposas Nerds', 0)
(2, 'Luciano Hashimoto', 'Raposas Nerds', 1)
(3, 'Rafael Montaigner', 'Raposas Nerds', 2)


Vemos apenas as linhas escolhidas. A operação de filtragem de linhas apropriadas chama-se ... **seleção**!!!

![Seleção](imgs/selecao.png)

Seleção e projeção são termos advindos da *álgebra relacional*, tópico que discutiremos em aulas futuras.

O comando `SELECT` também pode ser usado para efetuar cálculos:

In [18]:
db('SELECT 1 + 1')

Executando query:
(2,)


## Onde foram parar os dados?

MySQL mantém um grande conjunto de variáveis globais para que possamos verificar o estado do sistema. Podemos consultar essas variáveis usando SQL! Afinal, se queremos armazenar informação (neste caso, sobre o próprio sistema), nada melhor que um SGBD!

In [19]:
# db('SHOW VARIABLES')

Para descobrir o valor de uma variável específica, use a cláusula `WHERE`:

In [20]:
db('SHOW VARIABLES WHERE Variable_name = "version"')

Executando query:
('version', '8.0.30')


Para listar todas as variáveis com "dir" no nome:

In [21]:
db('SHOW VARIABLES WHERE Variable_name LIKE "%dir%"')

Executando query:
('basedir', 'C:\\Program Files\\MySQL\\MySQL Server 8.0\\')
('binlog_direct_non_transactional_updates', 'OFF')
('character_sets_dir', 'C:\\Program Files\\MySQL\\MySQL Server 8.0\\share\\charsets\\')
('datadir', 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Data\\')
('innodb_data_home_dir', '')
('innodb_directories', '')
('innodb_doublewrite_dir', '')
('innodb_log_group_home_dir', '.\\')
('innodb_max_dirty_pages_pct', '90.000000')
('innodb_max_dirty_pages_pct_lwm', '10.000000')
('innodb_redo_log_archive_dirs', '')
('innodb_temp_tablespaces_dir', '.\\#innodb_temp\\')
('innodb_tmpdir', '')
('innodb_undo_directory', '.\\')
('lc_messages_dir', 'C:\\Program Files\\MySQL\\MySQL Server 8.0\\share\\')
('plugin_dir', 'C:\\Program Files\\MySQL\\MySQL Server 8.0\\lib\\plugin\\')
('replica_load_tmpdir', 'C:\\WINDOWS\\SERVIC~1\\NETWOR~1\\AppData\\Local\\Temp')
('slave_load_tmpdir', 'C:\\WINDOWS\\SERVIC~1\\NETWOR~1\\AppData\\Local\\Temp')
('tmpdir', 'C:\\WINDOWS\\SERVIC~1\\NETWOR~1\\Ap

Observe a variável `datadir`: ela contém o diretório onde o MySQL armazena nossos dados. Na minha máquina é `C:\ProgramData\MySQL\MySQL Server 8.0\Data\`

<center><img src='imgs/datadir.png'/></center>

Note que esse diretório também tem várias chaves criptográficas! Uma vez que o acesso físico ao seu servidor for comprometido, você pode perder completamente o controle, pois o invasor agora tem até mesmo as chaves de acesso!

Dentro do diretório `torneio` temos nossa base de dados no disco:

<center><img src='imgs/torneio.png'/></center>


## Carregando dados em massa

Se você tem um arquivo com vários itens de dados, o melhor jeito de inserí-los no banco de dados é usar os comandos de carga de dados do seu SGBD preferido. No MySQL, veja a documentação do comando `LOAD DATA` em https://dev.mysql.com/doc/refman/8.0/en/load-data.html

Daqui para frente acostume-se a consultar a documentação de todo novo comando que você descobrir!

# `UPDATE`

Vamos alterar informações na nossa base. Para isso vamos usar o comando `UPDATE`. 

Suponha que desejamos alterar o grito de guerra da equipe "Raposas Nerds" de 'sudo vencer!' para 'sudo vencer --force!':

In [22]:
db('UPDATE Equipes SET grito="sudo vencer --force!" WHERE nome="Raposas Nerds"')

Executando query:


Verificando o resultado:

In [23]:
db('SELECT * from Equipes')

Executando query:
('É nois', 'Olha eu mamãe!')
('Raposas Nerds', 'sudo vencer --force!')


**Atividade:** Passe o "Rafael Montaigner" para o time "É nois".

In [24]:
db('UPDATE Jogadores SET nome_equipe="É nois" WHERE id="3"')

Executando query:


In [25]:
db('SELECT * from Jogadores')

Executando query:
(1, 'Raul Ayres', 'Raposas Nerds', 0)
(2, 'Luciano Hashimoto', 'Raposas Nerds', 1)
(3, 'Rafael Montaigner', 'É nois', 2)
(4, 'Igor Miranda', 'É nois', 2)
(5, 'Andrew Ikeda', 'É nois', 1)
(6, 'Fábio Kurauchi', 'É nois', 0)


**Atividade:** Tente mudar o nome de "Andrew Ikeda" para "Andrew Gomes da Silva". O que aconteceu?

In [26]:
db('UPDATE Jogadores SET nome="Andrew Gomes da Silva" WHERE id="5"')

Executando query:


In [27]:
db('SELECT * from Jogadores')

Executando query:
(1, 'Raul Ayres', 'Raposas Nerds', 0)
(2, 'Luciano Hashimoto', 'Raposas Nerds', 1)
(3, 'Rafael Montaigner', 'É nois', 2)
(4, 'Igor Miranda', 'É nois', 2)
(5, 'Andrew Gomes da Silva', 'É nois', 1)
(6, 'Fábio Kurauchi', 'É nois', 0)


**Atividade:** Tente agora mudar o nome do time "É nois" para "Somos nos". O que aconteceu?

In [28]:
db('UPDATE Equipes SET nome="Somos nós" WHERE nome="É nois"')

Executando query:


IntegrityError: 1451 (23000): Cannot delete or update a parent row: a foreign key constraint fails (`torneio`.`jogadores`, CONSTRAINT `fk_equipe` FOREIGN KEY (`nome_equipe`) REFERENCES `equipes` (`nome`))

1451 (23000): Cannot delete or update a parent row: a foreign key constraint fails (`torneio`.`jogadores`, CONSTRAINT `fk_equipe` FOREIGN KEY (`nome_equipe`) REFERENCES `equipes` (`nome`))

## `COMMIT` e `ROLLBACK`

Note que as ultimas atividades (de `UPDATE`) foram feitas sem executar `connection.commit()`. Neste caso todas as modificações realizadas ainda não foram registradas no banco de dados! Estas mudanças existem apenas na nossa *sessão*. Vamos verificar esse fenômeno. Primeiro vamos ver o estado das nossas tabelas:

In [29]:
db('SELECT * FROM Jogadores')

Executando query:
(1, 'Raul Ayres', 'Raposas Nerds', 0)
(2, 'Luciano Hashimoto', 'Raposas Nerds', 1)
(3, 'Rafael Montaigner', 'É nois', 2)
(4, 'Igor Miranda', 'É nois', 2)
(5, 'Andrew Gomes da Silva', 'É nois', 1)
(6, 'Fábio Kurauchi', 'É nois', 0)


In [30]:
db('SELECT * FROM Equipes')

Executando query:
('É nois', 'Olha eu mamãe!')
('Raposas Nerds', 'sudo vencer --force!')


Agora vamos verificar o estado do banco de dados lá no MySQL Workbench:

<center><img src='imgs/equipes.png'/></center>

<center><img src='imgs/jogadores.png'/></center>

Está diferente! Isso acontece porque as mudanças da nossa sessão ainda não foram *committed*, não foram registradas em definitivo. Para registrar as mudanças você pode executar o comando `connection.commit()`, ou equivalentemente rodar o comando SQL `COMMIT`. Se você se arrependeu das mudanças realizadas na sessão, jogue elas fora com o comando `connection.rollback()` ou equivalentemente o comando SQL `ROLLBACK`.

Vamos então concretizar nossas mudanças:

In [31]:
connection.commit()
# ou então db('COMMIT'), é a mesma coisa.

Agora vamos verificar lá no MySQL Workbench:

<center><img src='imgs/equipes2.png'/></center>

<center><img src='imgs/jogadores2.png'/></center>

Podemos constatar que as mudanças feitas na nossa sessão agora são visíveis na outra sessão (a do MySQL Workbench). Vamos revisitar futuramente os comandos `COMMIT` e `ROLLBACK` na aula de transações SQL.

## `DELETE`

Vamos remover o 'Fábio Kurauchi' da tabela `Jogadores`

In [32]:
try:
    db('DELETE FROM Jogadores WHERE nome="Fábio Kurauchi"')
    connection.commit()
except Exception as e:
    print(f'{type(e)}: {e}')
    connection.rollback()

Executando query:


Verificando o resultado:

In [33]:
db('SELECT * FROM Jogadores')

Executando query:
(1, 'Raul Ayres', 'Raposas Nerds', 0)
(2, 'Luciano Hashimoto', 'Raposas Nerds', 1)
(3, 'Rafael Montaigner', 'É nois', 2)
(4, 'Igor Miranda', 'É nois', 2)
(5, 'Andrew Gomes da Silva', 'É nois', 1)


**Atividade:** Remova da tabela de jogadores todos aqueles que preferem jogar cowboy.

In [34]:
db('DELETE FROM Jogadores WHERE preferencia="1"')

Executando query:


In [35]:
db('SELECT * FROM Jogadores')

Executando query:
(1, 'Raul Ayres', 'Raposas Nerds', 0)
(3, 'Rafael Montaigner', 'É nois', 2)
(4, 'Igor Miranda', 'É nois', 2)


Agora vamos tentar remover o time 'É nois':

In [36]:
try:
    db('DELETE FROM Equipes WHERE nome="É nois"')
    connection.commit()
except Exception as e:
    print(f'{type(e)}: {e}')
    connection.rollback()

Executando query:
<class 'mysql.connector.errors.IntegrityError'>: 1451 (23000): Cannot delete or update a parent row: a foreign key constraint fails (`torneio`.`jogadores`, CONSTRAINT `fk_equipe` FOREIGN KEY (`nome_equipe`) REFERENCES `equipes` (`nome`))


Novamente o malfadado erro `IntegrityError`. Isso acontece porque o SGBD está fazendo exatamente o que você pediu: garantindo a integridade dos dados! Afinal, se removermos a equipe `'É nois'`, o que fazemos com as linhas da tabela-filha que referenciam esse registro?

A restrição de chave estrangeira no nome da equipe (em `script_001.sql` ela foi chamada de `fk_equipe`) especifica, implicitamente, que a ação de apagar um registro deve ser bloqueada se impactar tabelas filhas. Podemos mudar essa restrição.

## `ON UPDATE / ON DELETE`

Vamos reescrever a restrição de chave estrangeira. Copie o script a seguir para `script_002.sql`.

```SQL
USE torneio;

ALTER TABLE Jogadores
    DROP FOREIGN KEY fk_equipe;

ALTER TABLE Jogadores
    ADD CONSTRAINT fk_equipe FOREIGN KEY (nome_equipe) REFERENCES Equipes (nome)
    ON DELETE CASCADE
    ON UPDATE CASCADE;

```

Feche a conexão atual com o banco de dados: ela está bloqueando alterações no schema. Não se esqueça de efetivar as modificações passadas com `commit()`, se elas não foram *committed* até o momento:

In [37]:
connection.commit()
connection.close()

Agora rode `script_002.sql` no MySQL Workbench. Se tudo deu certo você deve observar a mudança de ação na restrição de chave estrageira:

<center><img src='imgs/cascade.png'/></center>

Reabra a conexão e recrie o *helper*

In [38]:
connection = mysql.connector.connect(**connection_options)
db = ConnectionHelper(connection)

Vamos agora tentar mudar o nome da equipe 'É nois' para 'Somos nós':

In [39]:
db('UPDATE Equipes SET nome="Somos nós" WHERE nome="É nois"')

Executando query:


Verificando o resultado:

In [40]:
db('SELECT * from Equipes')

Executando query:
('Raposas Nerds', 'sudo vencer --force!')
('Somos nós', 'Olha eu mamãe!')


Vamos verificar o que aconteceu com a tabela Jogadores

In [41]:
db('SELECT * FROM Jogadores')

Executando query:
(1, 'Raul Ayres', 'Raposas Nerds', 0)
(2, 'Luciano Hashimoto', 'Raposas Nerds', 1)
(3, 'Rafael Montaigner', 'Somos nós', 2)
(4, 'Igor Miranda', 'Somos nós', 2)
(5, 'Andrew Gomes da Silva', 'Somos nós', 1)


Como você pode ver, a mudança de nome foi devidamente propagada, cortesia do `ON UPDATE CASCADE`.

Já o `ON DELETE CASCADE` é mais bruto. Vamos remover o time 'Somos nós':

In [42]:
db('DELETE FROM Equipes WHERE nome="Somos nós"')

Executando query:


In [43]:
db('SELECT * from Equipes')

Executando query:
('Raposas Nerds', 'sudo vencer --force!')


Ok. E os jogadores?

In [44]:
db('SELECT * FROM Jogadores')

Executando query:
(1, 'Raul Ayres', 'Raposas Nerds', 0)
(2, 'Luciano Hashimoto', 'Raposas Nerds', 1)


A ação `CASCADE` associada ao evento `ON DELETE` acabou por limpar a tabela `Jogadores` tambem.

Vamos cancelar todas essas ações com o comando `rollback()`, que é o oposto de `commit()`

In [45]:
connection.rollback()

Verificando se deu certo:

In [46]:
db('SELECT * FROM Jogadores')

Executando query:
(1, 'Raul Ayres', 'Raposas Nerds', 0)
(2, 'Luciano Hashimoto', 'Raposas Nerds', 1)
(3, 'Rafael Montaigner', 'É nois', 2)
(4, 'Igor Miranda', 'É nois', 2)
(5, 'Andrew Gomes da Silva', 'É nois', 1)


**Atividade:** Alem de `CASCADE` e `RESTRICT` existem outros especificadores de ação. Ache a URL da documentação que mostra essas opções.

**R:** https://dev.mysql.com/doc/refman/8.0/en/create-table-foreign-keys.html

## `REPLACE` = `DELETE` + `INSERT`

O comando `REPLACE` lembra um pouco o comando `UPDATE`, e é aí que mora o perigo! `REPLACE` é na verdade uma combinação de `DELETE` seguido de `INSERT`. Vamos exemplificar: suponha que eu quero atualizar o grito do time 'É nois' para 'Vai dar ruim!', mas ao invés de usar o `UPDATE` resolvi usar o `REPLACE`:

In [47]:
db('SELECT * from Equipes')
db('SELECT * from Jogadores')

Executando query:
('É nois', 'Olha eu mamãe!')
('Raposas Nerds', 'sudo vencer --force!')
Executando query:
(1, 'Raul Ayres', 'Raposas Nerds', 0)
(2, 'Luciano Hashimoto', 'Raposas Nerds', 1)
(3, 'Rafael Montaigner', 'É nois', 2)
(4, 'Igor Miranda', 'É nois', 2)
(5, 'Andrew Gomes da Silva', 'É nois', 1)


In [48]:
db('REPLACE INTO Equipes VALUES ("É nois", "Vai dar ruim!")')

Executando query:


In [49]:
db('SELECT * from Equipes')

Executando query:
('É nois', 'Vai dar ruim!')
('Raposas Nerds', 'sudo vencer --force!')


In [50]:
# Como tá no CASCADE, ele deleta e depois insere o é nois, e por isso tudo morreu yay
db('SELECT * from Jogadores')

Executando query:
(1, 'Raul Ayres', 'Raposas Nerds', 0)
(2, 'Luciano Hashimoto', 'Raposas Nerds', 1)


In [52]:
connection.rollback()
db('SELECT * from Equipes')
db('SELECT * from Jogadores')

Executando query:
('É nois', 'Olha eu mamãe!')
('Raposas Nerds', 'sudo vencer --force!')
Executando query:
(1, 'Raul Ayres', 'Raposas Nerds', 0)
(2, 'Luciano Hashimoto', 'Raposas Nerds', 1)
(3, 'Rafael Montaigner', 'É nois', 2)
(4, 'Igor Miranda', 'É nois', 2)
(5, 'Andrew Gomes da Silva', 'É nois', 1)


Aparentemente deu ruim!

**Atividade:** Explique

**R:**

Ufa, por hoje é só! Feche a conexão e até a próxima aula!

In [53]:
connection.close()